In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
from gen_data import get_data, fit

In [3]:
d = get_data(25)                                         # grab the data 
fits = [fit(m) for m in d]                               # fit all of the curves
fits_df = pd.DataFrame(fits, index=d.coords["control"])  # put the fit values is a DataFrame

In [4]:
fig, ax = plt.subplots()
fits_df.plot(y=['zeta', 'omega'], ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='control'>

In [5]:
def plot_zeta(ax, fits_df):
    ax.set_ylabel(r"$\zeta$")
    ax.set_xlabel("control (arb)")
    ax.set_ylim(0, 0.08)
    return ax.plot(
        fits_df["zeta"], marker="o", color="k", label="\N{greek small letter zeta}", linestyle=''
    )


def plot_omega(ax, fits_df):
    ax.set_ylabel(r"$\omega_0/2\pi$ (kHz)")
    ax.set_xlabel("control (arb)")
    ax.set_ylim(0, 1.25)
    return ax.plot(
        fits_df["omega"] / (2 * np.pi),
        marker="o",
        color="k",
        label="\N{greek small letter omega}",
    )


In [6]:
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True)
plot_zeta(ax1, fits_df)
plot_omega(ax2, fits_df)
ax1.legend()
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def plot_one(ax, m, popt, offset=0):
    # pull what we want out of the xarray
    control = float(m.coords["control"])
    t = m.coords["time"]
    z = m.values

    (ln,) = ax.plot(t, z + offset, label=f"C: {control:.1f}")
    (fit,) = ax.plot(t, popt.sample(t) + offset, color="k")
    ann = ax.annotate(
        f"$C={control:.1f}$\n$\\zeta={popt.zeta:.2g}$ $\\omega_0={popt.omega:.2f}$",
        # units are (axes-fraction, data)
        xy=(0.95, offset + 0.5),
        xycoords=ax.get_yaxis_transform(),
        # set the text alignment
        ha="right",
        va="bottom",
    )
    return {"raw": ln, "fit": fit, "annotation": ann}

def plot_several(ax, d, fits):
    out = []

    for j, (m, popt) in enumerate(zip(d, fits)):
        arts = plot_one(ax, m, popt, offset=3.75 * j)
        out.append(arts)

    ax.set_xlabel("time (ms)")
    ax.set_ylabel("displacement (mm)")

    return out

In [8]:
fig, ax = plt.subplots()
indx = [0, 10, 24]
plot_several(ax, d[indx], [fits[i] for i in indx])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[{'raw': <matplotlib.lines.Line2D at 0x7faf1e4e9b20>,
  'fit': <matplotlib.lines.Line2D at 0x7faf1a433d00>,
  'annotation': Text(0.95, 0.5, '$C=15.0$\n$\\zeta=0.036$ $\\omega_0=3.00$')},
 {'raw': <matplotlib.lines.Line2D at 0x7faf1e5377c0>,
  'fit': <matplotlib.lines.Line2D at 0x7faf1a4806d0>,
  'annotation': Text(0.95, 4.25, '$C=21.2$\n$\\zeta=0.028$ $\\omega_0=4.01$')},
 {'raw': <matplotlib.lines.Line2D at 0x7faf1e562ac0>,
  'fit': <matplotlib.lines.Line2D at 0x7faf1a466610>,
  'annotation': Text(0.95, 8.0, '$C=30.0$\n$\\zeta=0.045$ $\\omega_0=6.00$')}]

In [9]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, constrained_layout=True)
indx = [0, 10, 24]
plot_several(ax1, d[indx], [fits[i] for i in indx])
plot_zeta(ax2, fits_df)
plot_omega(ax3, fits_df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig, ax_dict = plt.subplot_mosaic("""
AB
AC
""", constrained_layout=True)
indx = [0, 10, 24]
plot_several(ax_dict['A'], d[indx], [fits[i] for i in indx])
plot_zeta(ax_dict['B'], fits_df)
plot_omega(ax_dict['C'], fits_df)
fig.align_ylabels(list(ax_dict.values()))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
def subplot_label(ax, text):
    return ax.annotate(
        text,
        # units are (axes-fraction, axes-fraction)
        # # this is bottom right
        # xy=(1, 0),
        # this is the top left
        xy=(0, 1),
        xycoords="axes fraction",
        # units are absolute offset in points from xy
        xytext=(-5, 5),
        textcoords=("offset points"),
        # set the text alignment
        ha="right",
        va="bottom",
        fontweight="bold",
        fontsize="larger",
    )


In [12]:
fig, ax_dict = plt.subplot_mosaic("""
AB
AC
""", constrained_layout=True)
indx = [0, 10, 24]
plot_several(ax_dict['A'], d[indx], [fits[i] for i in indx])
plot_zeta(ax_dict['B'], fits_df)
plot_omega(ax_dict['C'], fits_df)
fig.align_ylabels(list(ax_dict.values()))
subplot_labels = {
    k: subplot_label(v, f"({k.lower()})") for k, v in ax_dict.items()
}

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def paper_figure_2(fig, layout, d, fits, *, plot_every=5):
    ax_dict = fig.subplot_mosaic(layout)

    fits_df = pd.DataFrame(fits, index=d.coords["control"])

    index = list(range(0, len(d), plot_every))

    artists = {
        "vibrations": plot_several(ax_dict["A"], d[index], [fits[i] for i in index]),
        "zeta": plot_zeta(ax_dict["B"], fits_df),
        "omega": plot_omega(ax_dict["C"], fits_df),
    }

    fig.align_ylabels(list(ax_dict.values()))
    subplot_labels = {
        k: subplot_label(v, f"({k.lower()})") for k, v in ax_dict.items()
    }

    return (fig, ax_dict, artists, subplot_labels)

In [14]:
single_col_width = 8.6 / 2.54  # single column APS figure
double_col_width = 17.8 / 2.54  # double column APS figure


In [15]:
fig, axs, arts, labels = paper_figure_2(
    plt.figure(
        constrained_layout=True, figsize=(double_col_width, double_col_width * 0.5)
    ),
    "AB\nAC",
    d,
    fits,
    plot_every=5,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
paper_figure_2(
    plt.figure(
        constrained_layout=True, figsize=(single_col_width, single_col_width * 2.5)
    ),
    "A\nB\nC",
    d,
    fits,
    plot_every=10,
)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 338.583x846.457 with 3 Axes>,
 {'C': <AxesSubplot:label='C', xlabel='control (arb)', ylabel='$\\omega_0/2\\pi$ (kHz)'>,
  'B': <AxesSubplot:label='B', xlabel='control (arb)', ylabel='$\\zeta$'>,
  'A': <AxesSubplot:label='A', xlabel='time (ms)', ylabel='displacement (mm)'>},
 {'vibrations': [{'raw': <matplotlib.lines.Line2D at 0x7faf186ba880>,
    'fit': <matplotlib.lines.Line2D at 0x7faf186badf0>,
    'annotation': Text(0.95, 0.5, '$C=15.0$\n$\\zeta=0.036$ $\\omega_0=3.00$')},
   {'raw': <matplotlib.lines.Line2D at 0x7faf186c0310>,
    'fit': <matplotlib.lines.Line2D at 0x7faf186c0a30>,
    'annotation': Text(0.95, 4.25, '$C=21.2$\n$\\zeta=0.028$ $\\omega_0=4.01$')},
   {'raw': <matplotlib.lines.Line2D at 0x7faf186c0b20>,
    'fit': <matplotlib.lines.Line2D at 0x7faf186c71f0>,
    'annotation': Text(0.95, 8.0, '$C=27.5$\n$\\zeta=0.041$ $\\omega_0=5.36$')}],
  'zeta': [<matplotlib.lines.Line2D at 0x7faf186ba3a0>],
  'omega': [<matplotlib.lines.Line2D at 0x7faf186c78b0>]},